In [1]:
from neopipe.result import Result, Ok, Err
from neopipe.task import FunctionAsyncTask
from neopipe.async_pipeline import AsyncPipeline

In [2]:
@FunctionAsyncTask.decorator(retries=1)
async def add_one(res: Result[int, str]) -> Result[int, str]:
    if res.is_ok():
        return Ok(res.unwrap() + 1)
    return res

@FunctionAsyncTask.decorator(retries=1)
async def multiply_two(res: Result[int, str]) -> Result[int, str]:
    if res.is_ok():
        return Ok(res.unwrap() * 2)
    return res


In [9]:
async def test_run_parallel_success():
    p1 = AsyncPipeline.from_tasks([add_one, multiply_two])
    p2 = AsyncPipeline.from_tasks([multiply_two, add_one])
    inputs = [Ok(1), Ok(2)]
    res = await AsyncPipeline.run_parallel([p1, p2], inputs, debug=True)
    return res
    # assert res == Ok([[2, 4], [4, 5]])

In [10]:
result = await test_run_parallel_success()

2025-04-18 22:02:45 - neopipe.task - INFO - [add_one] Attempt 1 - Task ID: 70fe5576-79e4-4984-82fb-33337815451c
2025-04-18 22:02:45 - neopipe.task - INFO - [add_one] Success on attempt 1
2025-04-18 22:02:45 - neopipe.task - INFO - [multiply_two] Attempt 1 - Task ID: b54e5167-ef7c-41dd-a83e-dc5e24af873e
2025-04-18 22:02:45 - neopipe.task - INFO - [multiply_two] Success on attempt 1
2025-04-18 22:02:45 - neopipe.task - INFO - [multiply_two] Attempt 1 - Task ID: b54e5167-ef7c-41dd-a83e-dc5e24af873e
2025-04-18 22:02:45 - neopipe.task - INFO - [multiply_two] Success on attempt 1
2025-04-18 22:02:45 - neopipe.task - INFO - [add_one] Attempt 1 - Task ID: 70fe5576-79e4-4984-82fb-33337815451c
2025-04-18 22:02:45 - neopipe.task - INFO - [add_one] Success on attempt 1


In [11]:
result.unwrap()

([PipelineResult(name='AsyncPipeline-f91635fe-7428-4a8c-af32-e9d95380eaf4', result=4),
  PipelineResult(name='AsyncPipeline-ae60c73c-b043-469a-a679-d6cc3f464a2a', result=5)],
 PipelineTrace(pipelines=[SinglePipelineTrace(name='AsyncPipeline-f91635fe-7428-4a8c-af32-e9d95380eaf4', tasks=[('add_one', Ok(2)), ('multiply_two', Ok(4))]), SinglePipelineTrace(name='AsyncPipeline-ae60c73c-b043-469a-a679-d6cc3f464a2a', tasks=[('multiply_two', Ok(4)), ('add_one', Ok(5))])]))